**Magic Square**

A magic square is a square matrix where the order is odd, and the sum of the elements
in each row, column, and diagonal is the same. The sum of each row, column, or diagonal can be calculated using the formula: n(n^2 + 1)/2, where n is the order of the square.
For example, in a 3x3 magic square, the sum for each row, column, and diagonal should be 15.

This program generates and solves a classic 3x3 magic square puzzle called the "Moving Magic Square." The puzzle is played on a 3x3 table containing the numbers 1 to 9, with the number '9' being the "movable number." The movable number can be swapped with numbers in four directions: up, down, left, or right. The goal is to move the '9' to reach a final state where the sum of the three numbers in every row, column, and diagonal equals 15. There are multiple solutions that satisfy this condition, and the program should stop once a valid solution is found.

This program applies a Genetic Algorithm to solve the puzzle. The algorithm works by creating an initial population of random puzzle configurations and then using genetic operations like selection, crossover, and mutation to evolve towards a solution. The genetic algorithm should be able to generate random puzzle instances, which may have zero, one, or more solutions.

In [ ]:
import random

class MagicSquare:
    def __init__(self, population_size=4, mutation_rate=0.1, max_generations=1000):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.max_generations = max_generations

    def initialize_population(self):
        population = []
        for _ in range(self.population_size):
            puzzle = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
            numbers = list(range(1, 10))
            random.shuffle(numbers)
            for i in range(3):
                for j in range(3):
                    puzzle[i][j] = numbers.pop(0)
            population.append(puzzle)
        return population

    def row_sum(self, puzzle, row):
        return sum(puzzle[row])

    def col_sum(self, puzzle, col):
        return sum(puzzle[i][col] for i in range(3))

    def diag_sum1(self, puzzle):
        return sum(puzzle[i][i] for i in range(3))

    def diag_sum2(self, puzzle):
        return sum(puzzle[i][2-i] for i in range(3))

    def fitness(self, puzzle):
        target_sum = 15
        row_sums = [self.row_sum(puzzle, i) for i in range(3)]
        col_sums = [self.col_sum(puzzle, j) for j in range(3)]
        diag_sum1_val = self.diag_sum1(puzzle)
        diag_sum2_val = self.diag_sum2(puzzle)
        fitness_score = sum(1 for s in row_sums + col_sums + [diag_sum1_val, diag_sum2_val] if s == target_sum)
        return fitness_score

    def select_parents(self, population):
        sorted_population = sorted(population, key=self.fitness, reverse=True)
        if len(sorted_population) >= 2:
            return sorted_population[:2]
        else:
            return sorted_population[:1], sorted_population[:1]

    def crossover(self, parent1, parent2):
        if len(parent1) == len(parent2) == 3 and all(len(row) == 3 for row in parent1) and all(len(row) == 3 for row in parent2):
            crossover_point = random.randint(0, 2)
            child1 = [parent1[i][:crossover_point] + parent2[i][crossover_point:] for i in range(3)]
            child2 = [parent2[i][:crossover_point] + parent1[i][crossover_point:] for i in range(3)]
            return child1, child2
        else:
            raise ValueError("Invalid parent dimensions for crossover")

    def mutate(self, puzzle):
        if random.random() < self.mutation_rate:
            x, y = random.randint(0, 2), random.randint(0, 2)
            available_numbers = [num for num in range(1, 10) if num not in puzzle[x]]
            available_numbers = [num for num in available_numbers if num not in [puzzle[i][y] for i in range(3)]]
            box_row, box_col = x // 3 * 3, y // 3 * 3
            available_numbers = [num for num in available_numbers if num not in [puzzle[box_row + i // 3][box_col + i % 3] for i in range(9)]]
            if available_numbers:
                puzzle[x][y] = random.choice(available_numbers)
        return puzzle

    def solve(self):
        population = self.initialize_population()
        for generation in range(self.max_generations):
            parent1, parent2 = self.select_parents(population)
            child1, child2 = self.crossover(parent1, parent2)
            child1 = self.mutate(child1)
            child2 = self.mutate(child2)
            population.extend([child1, child2])
            population = sorted(population, key=self.fitness, reverse=True)[:self.population_size]
            if self.fitness(population[0]) == 9:
                return population[0], generation
        return population[0], self.max_generations

matrix = MagicSquare()
solution, generations = matrix.solve()
print(matrix.initialize_population())
if solution:
    print("Solution found in generation:", generations)
    print("Magic Square:")
    for row in solution:
        print(row)
else:
    print("No solution found within the maximum number of generations.")


**Sudoku Solver**

Sudoku, a Japanese word meaning "single numbers," is a popular numerical puzzle game. It is often found in newspapers and gaming publications around the world. While the rules of the game are simple, the solutions can range from very simple to extremely challenging. In this assignment, we are tasked with developing a Sudoku solver using search algorithms with heuristics to solve the puzzle.

The classic 9x9 Sudoku puzzle consists of a 9x9 grid, with some cells re-filled with digits and other cells left empty. The goal is to fill in the empty cells such that:
1. Each row must contain exactly one of each digit from 1 to 9.
2. Each column must contain exactly one of each digit from 1 to 9.
3. Each 3x3 box (sub-grid) must contain exactly one of each digit from 1 to 9.

The Sudoku solver should apply search algorithms and heuristics to efficiently find solutions to the puzzle. The search algorithms will explore possible solutions by filling in empty cells while maintaining the constraints mentioned above.

The solver should applies a heuristic to minimize the search space and prioritize filling in cells that have fewer possible candidates, reducing the number of possibilities at each step.



In [ ]:
class SudokuSolver:
    def __init__(self, puzzle):
        self.puzzle = puzzle
        self.n = len(puzzle)
        self.empty_cell = 0

    def solve(self):
        if self.backtracking_search():
            return self.puzzle  # Return the solved puzzle
        else:
            return None

    def backtracking_search(self):
        if not self.has_empty_cell():
            return True  # Puzzle solved

        row, col = self.select_unassigned_variable()

        for value in self.order_values(row, col):
            if self.is_valid_assignment(row, col, value):
                self.puzzle[row][col] = value

                if self.backtracking_search():
                    return True

                self.puzzle[row][col] = self.empty_cell  # Backtrack

        return False

    def select_unassigned_variable(self):
        min_remaining_values = float('inf')
        selected_row, selected_col = -1, -1

        for row in range(self.n):
            for col in range(self.n):
                if self.puzzle[row][col] == self.empty_cell:
                    remaining_values = len(self.get_available_values(row, col))

                    if remaining_values < min_remaining_values:
                        min_remaining_values = remaining_values
                        selected_row, selected_col = row, col

                    elif remaining_values == min_remaining_values:
                        if self.degree_heuristic(row, col) > self.degree_heuristic(selected_row, selected_col):
                            selected_row, selected_col = row, col

        return selected_row, selected_col

    def order_values(self, row, col):
        available_values = self.get_available_values(row, col)
        return sorted(available_values, key=lambda x: self.least_constraining_value(row, col, x))

    def get_available_values(self, row, col):
        values = set(range(1, self.n + 1))
        values -= set(self.puzzle[row])
        values -= set(self.puzzle[i][col] for i in range(self.n))
        box_row, box_col = row // 3 * 3, col // 3 * 3
        values -= set(self.puzzle[box_row + i][box_col + j] for i in range(3) for j in range(3))

        return values

    def least_constraining_value(self, row, col, value):
        conflicts = 0

        for i in range(self.n):
            if self.puzzle[row][i] == self.empty_cell and value in self.get_available_values(row, i):
                conflicts += 1

            if self.puzzle[i][col] == self.empty_cell and value in self.get_available_values(i, col):
                conflicts += 1

            box_row, box_col = row // 3 * 3, col // 3 * 3

            if self.puzzle[box_row + i // 3][box_col + i % 3] == self.empty_cell and value in self.get_available_values(box_row + i // 3, box_col + i % 3):
                conflicts += 1

        return conflicts

    def degree_heuristic(self, row, col):
        # Count the number of empty cells in the same row, column, and box
        count = 0

        for i in range(self.n):
            if self.puzzle[row][i] == self.empty_cell:
                count += 1

            if self.puzzle[i][col] == self.empty_cell:
                count += 1

            box_row, box_col = row // 3 * 3, col // 3 * 3

            if self.puzzle[box_row + i // 3][box_col + i % 3] == self.empty_cell:
                count += 1

        return count

    def has_empty_cell(self):
        return any(self.empty_cell in row for row in self.puzzle)

    def is_valid_assignment(self, row, col, value):
        return (self.is_valid_row(row, value) and
                self.is_valid_col(col, value) and
                self.is_valid_box(row, col, value))

    def is_valid_row(self, row, value):
        return value not in self.puzzle[row]

    def is_valid_col(self, col, value):
        return value not in [self.puzzle[i][col] for i in range(self.n)]

    def is_valid_box(self, row, col, value):
        box_row, box_col = row // 3 * 3, col // 3 * 3
        return value not in [self.puzzle[box_row + i // 3][box_col + i % 3] for i in range(9)]

puzzle = [
        [0, 6, 0, 1, 0, 4, 0, 5, 0],
        [0, 0, 8, 3, 0, 5, 6, 0, 0],
        [2, 0, 0, 0, 0, 0, 0, 0, 1],
        [8, 0, 0, 4, 0, 7, 0, 0, 6],
        [0, 0, 6, 0, 0, 0, 3, 0, 0],
        [7, 0, 0, 9, 0, 1, 0, 0, 4],
        [5, 0, 0, 0, 0, 0, 0, 0, 2],
        [0, 0, 7, 2, 0, 6, 9, 0, 0],
        [0, 4, 0, 5, 0, 8, 0, 7, 0]
    ]

solver = SudokuSolver(puzzle)
solution = solver.solve()

if solution:
   print("Sudoku puzzle solved successfully:")
   for row in solution:
       print(row)
else:
   print("No solution found for the Sudoku puzzle.")



In [ ]:
import random

class SudokuGenerator:
    def __init__(self):
        self.grid = [[0 for _ in range(9)] for _ in range(9)]
        self.solve()

    def solve(self):
        for i in range(9):
            for j in range(9):
                if self.grid[i][j] == 0:
                    for num in range(1, 10):
                        if self.is_valid(i, j, num):
                            self.grid[i][j] = num
                            if self.solve():
                                return True
                            self.grid[i][j] = 0
                    return False
        return True

    def is_valid(self, row, col, num):
        for i in range(9):
            if self.grid[row][i] == num or self.grid[i][col] == num:
                return False
        start_row, start_col = 3 * (row // 3), 3 * (col // 3)
        for i in range(3):
            for j in range(3):
                if self.grid[start_row + i][start_col + j] == num:
                    return False
        return True

    def generate_puzzle(self):
        filled_cells = random.sample(range(81), 45)
        for cell in filled_cells:
            row, col = cell // 9, cell % 9
            self.grid[row][col] = 0

        return self.grid

generator = SudokuGenerator()
puzzle = generator.generate_puzzle()

print("Generated Sudoku puzzle:")
for row in puzzle:
    print(row)

solver = SudokuSolver(puzzle)
solution = solver.solve()

if solution:
   print("Sudoku puzzle solved successfully:")
   for row in solution:
       print(row)
else:
   print("No solution found for the Sudoku puzzle.")

Generated Sudoku puzzle:
[1, 0, 0, 0, 5, 0, 7, 8, 0]
[4, 0, 6, 0, 0, 9, 0, 2, 0]
[0, 8, 9, 0, 2, 3, 0, 5, 6]
[2, 1, 0, 0, 6, 5, 8, 9, 7]
[3, 6, 0, 8, 0, 0, 2, 0, 4]
[0, 9, 0, 0, 0, 4, 0, 0, 0]
[5, 0, 0, 0, 0, 0, 9, 0, 8]
[0, 4, 2, 9, 0, 0, 0, 0, 0]
[0, 0, 8, 0, 0, 0, 0, 0, 2]
Sudoku puzzle solved successfully:
[1, 2, 3, 4, 5, 6, 7, 8, 9]
[4, 5, 6, 7, 8, 9, 1, 2, 3]
[7, 8, 9, 1, 2, 3, 4, 5, 6]
[2, 1, 4, 3, 6, 5, 8, 9, 7]
[3, 6, 5, 8, 9, 7, 2, 1, 4]
[8, 9, 7, 2, 1, 4, 3, 6, 5]
[5, 3, 1, 6, 7, 2, 9, 4, 8]
[6, 4, 2, 9, 3, 8, 5, 7, 1]
[9, 7, 8, 5, 4, 1, 6, 3, 2]
